In [ ]:
#####################################################################################
# Audio-driven upper-body motion synthesis on a humanoid robot
# Computer Science Tripos Part III Project
# Jan Ondras (jo356@cam.ac.uk), Trinity College, University of Cambridge
# 2017/18
#####################################################################################
# Simulation of 3D pose reconstructed by the 3D pose estimation method OP+A
# See below for implementation of domain-specific assumptions (A)
#####################################################################################

In [15]:
#######################################################################################################
# Using 2D Openpose joints' locations AND constraints/assumptions => Lift to 3D
# Save as 1 matrix: #frames x #joints x 3 for each VID
# Done for all VIDs
#######################################################################################################
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import savemat
import json
import os
import glob
from geo import *

# Load 2D data for one VID 
#VID = 'PID20Task2'

# openpose_COCO_joints = 
# 	      {0,  "Nose"},
#         {1,  "Neck"},
#         {2,  "RShoulder"},
#         {3,  "RElbow"},
#         {4,  "RWrist"},
#         {5,  "LShoulder"},
#         {6,  "LElbow"},
#         {7,  "LWrist"},
#         {8,  "RHip"},
#         {9,  "RKnee"},
#         {10, "RAnkle"},
#         {11, "LHip"},
#         {12, "LKnee"},
#         {13, "LAnkle"},
#            	{14, "REye"},
#           	{15, "LEye"},
#           	{16, "REar"},
#           	{17, "LEar"},
#           	{18, "Background"} # not included in JSON files => 18 joints in total

N_joints = 18
# joints_used = [0,1,2,3,4,5,6,7,8, 11] # first 14 from COCO model, w/o legs => 10
jointID = {     # Mapping: joint names -> position in array as extracted
    'RHip'      : 8,    
    'LHip'      : 11,   
    'Neck'      : 1,
    'HeadTop'   : 0,  
    'LShoulder' : 5,
    'LElbow'    : 6,
    'LWrist'    : 7,
    'RShoulder' : 2,
    'RElbow'    : 3,
    'RWrist'    : 4
}

cnt = 0
for dir_path in sorted(glob.glob('./../Dataset/OpenposePoseFeatures/*')):

    VID = dir_path.split('/')[-1]

    json_filenames = sorted(glob.glob('./../Dataset/OpenposePoseFeatures/' + VID + '/*.json'))
    # print (json_filenames[0])
    # print (json_filenames[-1])
    print( 'VID:', VID, ',\t #frames: ', len(json_filenames)) # 1302
    N_frames = len(json_filenames)

    # Array for this VID: #frames x #joints x 3
    joints_3D = np.zeros( (N_frames, N_joints, 3) )

    # Limb lengths constants (might be adaptively updated if longer found)
    C1 = 0. 
    R2 = 0. 
    R3 = 0. 
    L2 = 0. 
    L3 = 0. 

    # Load: iterate over all frames IN ORDER ! IT IS IMPORTANT THAT IT IS SORTED !!!
    for i, json_filename in enumerate(json_filenames): 
        data = json.load(open(json_filename))
        if len(data['people']) != 1:
            print ("Error: multiple people detected, in ", json_filename) # but it's ok, always first one is the correct person detected - I checked manually

        f = np.reshape(data['people'][0]['pose_keypoints'], (-1,3)) # matrix = #joints x 3 (x,y,score)
        # print (len(f))
        # print (f)
        # f = f[joints_used] # keeping only 14 joints
        joints_3D[i, :, :2] = f[:, [0,1]] # copy only x, y, omit score; array joints_3D has now z=0 for all joints

        #################################################
        # Update (if necessary) current estimates of limb lengths
        C1m = Point(joints_3D[i, jointID['Neck']]).distance_to(Point(joints_3D[i, jointID['HeadTop']])) # measured in current frame
        C1 = max(C1, C1m) # maximum sofar - assumed to be actual limb length

        R2m = Point(joints_3D[i, jointID['RShoulder']]).distance_to(Point(joints_3D[i, jointID['RElbow']])) # measured in current frame
        R2 = max(R2, R2m) # maximum sofar - assumed to be actual limb length

        R3m = Point(joints_3D[i, jointID['RElbow']]).distance_to(Point(joints_3D[i, jointID['RWrist']])) # measured in current frame
        R3 = max(R3, R3m) # maximum sofar - assumed to be actual limb length

        L2m = Point(joints_3D[i, jointID['LShoulder']]).distance_to(Point(joints_3D[i, jointID['LElbow']])) # measured in current frame
        L2 = max(L2, L2m) # maximum sofar - assumed to be actual limb length

        L3m = Point(joints_3D[i, jointID['LElbow']]).distance_to(Point(joints_3D[i, jointID['LWrist']])) # measured in current frame
        L3 = max(L3, L3m) # maximum sofar - assumed to be actual limb length

        #################################################
        # Lift z coordinates of joints: HeadTop, RElbow, RWrist, LElbow, LWrist; others z = 0

        joints_3D[i, jointID['HeadTop'], 2] = np.sqrt( (C1**2) - (C1m**2) )

        joints_3D[i, jointID['RElbow'],  2] = np.sqrt( (R2**2) - (R2m**2) )
        joints_3D[i, jointID['LElbow'],  2] = np.sqrt( (L2**2) - (L2m**2) )

        joints_3D[i, jointID['RWrist'],  2] = joints_3D[i, jointID['RElbow'],  2] + np.sqrt( (R3**2) - (R3m**2) )
        joints_3D[i, jointID['LWrist'],  2] = joints_3D[i, jointID['LElbow'],  2] + np.sqrt( (L3**2) - (L3m**2) )

        joints_3D[i, :, 2] *= -1. # correct orientation


    #######################################################################################################
    # Save 3D joints, one file per VID

    np.savez('./../Dataset/OpenPoseConstraintsFeatures/' +VID+ '.npz', joints_3D=joints_3D)

    cnt += 1

print ("Done for all: ", cnt)

VID: PID02Task2 ,	 #frames:  3815
VID: PID02Task3 ,	 #frames:  3205
VID: PID05Task2 ,	 #frames:  3810
VID: PID05Task3 ,	 #frames:  3914
VID: PID06Task2 ,	 #frames:  2947
VID: PID06Task3 ,	 #frames:  3086
VID: PID08Task2 ,	 #frames:  3275
VID: PID08Task3 ,	 #frames:  3118
VID: PID09Task2 ,	 #frames:  3484
VID: PID09Task3 ,	 #frames:  3244
VID: PID10Task2 ,	 #frames:  2458
VID: PID10Task3 ,	 #frames:  2779
VID: PID11Task2 ,	 #frames:  2424
VID: PID11Task3 ,	 #frames:  3392
VID: PID13Task2 ,	 #frames:  2482
VID: PID13Task3 ,	 #frames:  1487
VID: PID15Task2 ,	 #frames:  2344
VID: PID15Task3 ,	 #frames:  3846
VID: PID16Task2 ,	 #frames:  2014
VID: PID16Task3 ,	 #frames:  1361
VID: PID17Task2 ,	 #frames:  2602
VID: PID17Task3 ,	 #frames:  2770
VID: PID18Task2 ,	 #frames:  2344
VID: PID18Task3 ,	 #frames:  2936
VID: PID19Task2 ,	 #frames:  2554
VID: PID19Task3 ,	 #frames:  2652
VID: PID20Task2 ,	 #frames:  1302
VID: PID20Task3 ,	 #frames:  2946
VID: PID21Task2 ,	 #frames:  1753
VID: PID21Task

In [1]:
#########################################################################################################
# Render in 3D
# and show on robot
#########################################################################################################

from geoutils import radToDeg, xyz_to_angles
from vpython import *
from scipy.io import loadmat
import numpy as np
from geo import *

VID = 'PID20Task2'
VID = 'PID23Task3'

STOP_AT_FRAMES = [1360, 1375, 1390]
STOP_AT_FRAMES = [1375, 1390]
STOP_AT_FRAMES = [1390]
STOP_AT_FRAMES = [1]

show_labels = True
# show_labels = False

# smooth_joints_pos = True # not done 
smooth_joints_pos = False

FR = 1000
FR = 10

#################################################
showRobot = True # simultaneously send commands to the robot
showRobot = False

angles_names = [
    "HeadPitch", "HeadYaw", 
    "LShoulderRoll", "LShoulderPitch", "LElbowRoll", "LElbowYaw",
    "RShoulderRoll", "RShoulderPitch", "RElbowRoll", "RElbowYaw", 
    "HipRoll", "HipPitch"
]
if showRobot:
    print ("Showing on virtual robot ...")
    from naoqi import ALProxy
    IP = "127.0.0.1"
    port = 45637
    motionProxy = ALProxy("ALMotion", IP, port)
    # Reset robot to neutral pose
    for an in angles_names:
        angle_reset = 0.
        if an == 'LShoulderPitch' or an == 'RShoulderPitch':
            angle_reset = angle_reset + np.pi/2
        motionProxy.setAngles(an, angle_reset, 1.)
        
    jointID = {     # Mapping: joint names -> position in array as extracted
        #'Pelvis'    : NA,  # not available => have to estimate from hips
        'RHip'      : 8,    
        'LHip'      : 11,   
        #'Spine'     : NA,  # not available, not used
        'Neck'      : 1,
        #'Nose'      : NA,  # not available => have to estimate HeadYaw another way
        'HeadTop'   : 0,  
        'LShoulder' : 5,
        'LElbow'    : 6,
        'LWrist'    : 7,
        'RShoulder' : 2,
        'RElbow'    : 3,
        'RWrist'    : 4
    }
#################################################

# joints_used = np.arange(17) # first 17
joints_used = [0,1,2,3,4,5,6,7, 8, 11] # 14 from COCO model, w/o legs, eyes, ears

joints_names = ['head_top','neck','right_shoulder','right_elbow','right_wrist',
                'left_shoulder','left_elbow','left_wrist',
                'right_hip','right_knee','right_ankle',
                'left_hip','left_knee','left_ankle',
                'right_eye','left_eye','right_ear', 'left_ear']
joint_connections = [
    (0, 1),                                                          # central, blue
    (1,8), (1,11),                                                   # extra for Openpose
    (1, 2), (2, 3), (3, 4), (8, 9), (9, 10), (0, 14), (14, 16),      # right, red
    (1, 5), (5, 6), (6, 7), (11, 12), (12, 13), (0, 15), (15, 17)    # left, green
]
joint_connections_colors = [color.blue]*3 + [color.red]*7 + [color.green]*7
joints_colors = [color.white]*len(joints_names)

if smooth_joints_pos:
#     data = np.load('./../Dataset/OpenPoseConstraintsFeatures_smoothed/' + VID + '.npz')['joints_3D']
    print "TODO"
    pass
else:
    data = np.load('./../Dataset/OpenPoseConstraintsFeatures/' + VID + '.npz')['joints_3D']

print ("Shape =" , np.shape(data))

# Setup scene
scene = canvas()
scene.width = 960
scene.height = 600
scene.title = "Openpose 3D pose reconstruction using Constraints and assumptions"
scene.scale = 400.
# scene.fov = np.pi /3.
scene.up=vector(0.,-1.,0.)     # To flip scene horiznotally !!!
scene.forward=vector(0.,0.,1.)     # To rotate around y !!!
# scene.center=vector(0.,-150.,10000) # To shift down a but

###################################### Centering to neck joint
data[0, :, :] = data[0, :, :] - data[0, 1, :]
# data[:,:,2] = 0. # if you remove depth

scene.lights = []
scene.ambient=color.gray(0.8)

# Texture uderneath
y_plane_pos = 500
plane_size = 1000
# a = vertex( pos=vector(-plane_size, y_plane_pos, -plane_size) )
# b = vertex( pos=vector(-plane_size, y_plane_pos,  plane_size) )
# c = vertex( pos=vector( plane_size, y_plane_pos,  plane_size) )
# d = vertex( pos=vector( plane_size, y_plane_pos, -plane_size) )
# quad(vs=[a,b,c,d], texture=textures.rug)

# SHOW PLANE UNDERNEATH THE FIGURE
#box(pos=vec(0,y_plane_pos,0), length=plane_size, height=1, width=plane_size, texture=textures.rough)

frame = 0
scene.caption = 'Frame: ' + str(frame)
Ps = []
Es = []
Ls = []
# Draw all joints requested
for i in joints_used:
    x = data[frame, i, 0] #- skeleton_center[0]
    y = data[frame, i, 1] #- skeleton_center[1]
    z = data[frame, i, 2] #- skeleton_center[2]
    P = sphere(pos=vector(x,y,z), radius=10, color=joints_colors[i])#, #size_units="world")
    Ps.append(P)
    if show_labels:
        L = label(pos=vector(x,y,z),
                text=joints_names[i], xoffset=20,
                yoffset=50, space=30,
                height=16, border=3,
                font='sans')
        Ls.append(L)

# Draw all links between joints
for i, (a,b) in enumerate(joint_connections):
    if a in joints_used and b in joints_used:
        ax = data[frame, a, 0] #- skeleton_center[0]
        ay = data[frame, a, 1] #- skeleton_center[1]
        az = data[frame, a, 2] #- skeleton_center[2]
        bx = data[frame, b, 0] #- skeleton_center[0]
        by = data[frame, b, 1] #- skeleton_center[1]
        bz = data[frame, b, 2] #- skeleton_center[2]
        cx = (ax + bx) / 2.
        cy = (ay + by) / 2.
        cz = (az + bz) / 2.
        E = ellipsoid(pos=vector(cx,cy,cz), axis=vector(ax-bx,ay-by,az-bz), length=np.linalg.norm([ax-bx,ay-by,az-bz]), 
                  height=25, width=25, color=joint_connections_colors[i])
        Es.append(E)


# Animate
for frame in range(1, len(data)):
    
    ####################################################################
    # Calculate joint angles & Send commands to the robot
    if showRobot:
        xyz = data[frame]
        ####################################################################
        # Calculate 12 joint angles, if not possible for this method set NAN (np.nan)
        # xyz[jointID['Pelvis']] gives xyz coordinates of Pelvis joint
        # For OP+C, pass None for Nose position => can only estimate HeadYaw
        # And estimate Pelvis position as midpoint of LHip-RHip
        PE = Point(xyz[jointID['LHip']]).midpoint_to(Point(xyz[jointID['RHip']]))

        jas = xyz_to_angles(Point(xyz[jointID['HeadTop']]), None, 
                            Point(xyz[jointID['Neck']]),    PE, 
                    Point(xyz[jointID['LShoulder']]), Point(xyz[jointID['LElbow']]), Point(xyz[jointID['LWrist']]), 
                    Point(xyz[jointID['RShoulder']]), Point(xyz[jointID['RElbow']]), Point(xyz[jointID['RWrist']])                  
        )
        
        for ang_ind, ang in enumerate(jas):
            if not np.isnan(ang): # take only angles available for this method
                motionProxy.setAngles(angles_names[ang_ind], ang, 1.)
    
    ###################################### Centering to neck joint
    data[frame, :, :] = data[frame, :, :] - data[frame, 1, :]
    
    rate(FR)
    scene.caption = 'Frame: ' + str(frame)
    # Draw all joints requested
    cnt = 0
    for i in joints_used:
        x = data[frame, i, 0] #- skeleton_center[0]
        y = data[frame, i, 1] #- skeleton_center[1]
        z = data[frame, i, 2] #- skeleton_center[2]
        Ps[cnt].pos = vector(x,y,z)
        if show_labels:
            Ls[cnt].pos = vector(x,y,z)
        cnt += 1

    # Draw all links between joints
    cnt = 0
    for i, (a,b) in enumerate(joint_connections):
        if a in joints_used and b in joints_used:
            ax = data[frame, a, 0] #- skeleton_center[0]
            ay = data[frame, a, 1] #- skeleton_center[1]
            az = data[frame, a, 2] #- skeleton_center[2]
            bx = data[frame, b, 0] #- skeleton_center[0]
            by = data[frame, b, 1] #- skeleton_center[1]
            bz = data[frame, b, 2] #- skeleton_center[2]    
            cx = (ax + bx) / 2.
            cy = (ay + by) / 2.
            cz = (az + bz) / 2.
            Es[cnt].pos = vector(cx,cy,cz)
            Es[cnt].axis = vector(ax-bx,ay-by,az-bz)
            Es[cnt].length = np.linalg.norm([ax-bx,ay-by,az-bz])
            cnt += 1

    if frame in STOP_AT_FRAMES:
        break

Shape = (2516, 18, 3)


<IPython.core.display.Javascript object>